In [1]:
# import numpy as np
# import pandas as pd
# import cv2
# import os
# from sklearn.svm import SVC
# from sklearn.preprocessing import StandardScaler, MinMaxScaler
# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import classification_report, accuracy_score, roc_auc_score
# from sklearn.linear_model import LogisticRegression
# from tensorflow.keras.applications import ResNet50
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, BatchNormalization, Dropout
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
# from tensorflow.keras.applications import EfficientNetB0
# from imblearn.over_sampling import RandomOverSampler
# from imblearn.over_sampling import SMOTE

# IMG_SIZE = (224, 224)  
# BATCH_SIZE = 32
# RANDOM_STATE = 42

# IMAGES_FOLDER = "/kaggle/input/glaucoma-datasets/G1020/Images"
# df = pd.read_csv("/kaggle/input/glaucoma-datasets/G1020/G1020.csv")

# def load_data(df, IMG_SIZE):
#     images = []
#     labels = []
    
#     for _, row in df.iterrows():
#         img_name = row['imageID']
#         label = row['binaryLabels']
#         img_path = os.path.join(IMAGES_FOLDER, img_name)
#         if os.path.exists(img_path):
#             img = cv2.imread(img_path)
#             img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#             img = cv2.resize(img, IMG_SIZE)
#             images.append(img)
#             labels.append(row['binaryLabels'])
    
#     return np.array(images), np.array(labels)

# images, labels = load_data(df, IMG_SIZE)
# images = images.astype('float32') / 255.0

# X_train, X_test, y_train, y_test = train_test_split(
#     images, labels, test_size=0.2, stratify=labels, random_state=RANDOM_STATE
# )

# datagen = ImageDataGenerator(
#     rotation_range=30,
#     brightness_range=[0.6, 1.4], 
#     zoom_range=0.3,
#     horizontal_flip=True
# )


# augmented_images = []
# augmented_labels = []
# for x, y in zip(X_train, y_train):
#     for _ in range(3):  
#         augmented_images.append(datagen.random_transform(x))
#         augmented_labels.append(y)
        
# X_train_aug = np.concatenate([X_train, np.array(augmented_images, dtype=np.float32)])
# y_train_aug = np.concatenate([y_train, np.array(augmented_labels)])


# # Feature extraction with ResNet50
# def extract_features(images):
#     # base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(*IMG_SIZE, 3))
#     # x = GlobalAveragePooling2D()(base_model.output)
#     # x = Dense(1024, activation='relu')(x)  
#     # x = Dense(512, activation='relu')(x)  
#     # x = Dropout(0.3)(x)
#     # feature_extractor = Model(inputs=base_model.input, outputs=x)

#     base_model = EfficientNetB0(weights="imagenet", include_top=False, input_shape=(*IMG_SIZE, 3))
#     x = GlobalAveragePooling2D()(base_model.output)
#     x = Dense(1024, activation='relu')(x)
#     x = Dropout(0.3)(x)
#     feature_extractor = Model(inputs=base_model.input, outputs=x)
    
#     features = feature_extractor.predict(images, batch_size=BATCH_SIZE, verbose=1)
#     return features


# print("Extracting training features...")
# train_features = extract_features(X_train)

# print("Extracting test features...")
# test_features = extract_features(X_test)

# # Normalize features
# # scaler = StandardScaler()
# scaler = MinMaxScaler()
# train_features_aug = extract_features(X_train_aug)
# X_train_features_aug = scaler.fit_transform(train_features_aug)
# X_test_features = scaler.transform(test_features)

# smote = SMOTE(random_state=RANDOM_STATE)
# X_train_sm, y_train_sm = smote.fit_resample(X_train_features_aug, y_train_aug)
# ros = RandomOverSampler(random_state=RANDOM_STATE)
# X_train_bal, y_train_bal = ros.fit_resample(X_train_features_aug, y_train_aug)

# # svm_params = {'C': [0.1, 1, 10], 'gamma': [0.01, 0.1, 1], 'kernel': ['rbf']}
# # svm_grid = GridSearchCV(SVC(probability=True), svm_params, cv=5, n_jobs=-1, verbose=2)
# # svm_grid.fit(X_train_bal, y_train_bal)
# # print(f"Best SVM parameters: {svm_grid.best_params_}")
# # y_pred = svm_grid.predict(X_test_features)
# # y_prob = svm_grid.predict_proba(X_test_features)[:, 1]

# svm = SVC(probability=True, class_weight={0: 1, 1: 3}, C=10, gamma=1, kernel='rbf')
# svm.fit(X_train_bal, y_train_bal)
# y_pred = svm.predict(X_test_features)
# y_prob = svm.predict_proba(X_test_features)[:, 1]

# rf = RandomForestClassifier(n_estimators=300, max_depth=20, class_weight={0: 1, 1: 3}, random_state=RANDOM_STATE)
# rf.fit(X_train_bal, y_train_bal)
# y_pred_rf = rf.predict(X_test_features)
# y_prob_rf = rf.predict_proba(X_test_features)[:, 1]

# cnn_model = Sequential([
#     Conv2D(32, (3,3), activation='relu', input_shape=(*IMG_SIZE, 3)),
#     MaxPooling2D(2,2),
#     Conv2D(64, (3,3), activation='relu'),
#     MaxPooling2D(2,2),
#     Flatten(),
#     Dense(512, activation='relu'),
#     Dropout(0.5),
#     Dense(1, activation='sigmoid')
# ])

# cnn_model.compile(optimizer='adam', loss=focal_loss(), metrics=['accuracy'])
# cnn_model.fit(X_train_bal, y_train_bal, epochs=10, batch_size=32)

# y_pred_cnn = cnn.predict(X_test_features)
# y_prob_cnn = cnn.predict_proba(X_test_features)[:, 1]

# print("\nClassification Report (SVM):")
# print(classification_report(y_test, y_pred, target_names=["Normal", "Glaucoma"]))
# print(f"\nSVM Accuracy: {accuracy_score(y_test, y_pred):.4f}")
# print(f"SVM AUC-ROC: {roc_auc_score(y_test, y_prob):.4f}")

# logreg = LogisticRegression(max_iter=1000)
# logreg.fit(X_train_features_aug, y_train_aug)
# y_pred_logreg = logreg.predict(X_test_features)
# print(f"Logistic Regression Accuracy: {accuracy_score(y_test, y_pred_logreg):.4f}")

# print("\nClassification Report (Random Forest):")
# print(classification_report(y_test, y_pred_rf, target_names=["Normal", "Glaucoma"]))
# print(f"\nRandom Forest Accuracy: {accuracy_score(y_test, y_pred_rf):.4f}")
# print(f"Random Forest AUC-ROC: {roc_auc_score(y_test, y_prob_rf):.4f}")

# print("\nClassification Report (CNN):")
# print(classification_report(y_test, y_pred_cnn, target_names=["Normal", "Glaucoma"]))
# print(f"\nCNN Accuracy: {accuracy_score(y_test, y_pred_cnn):.4f}")
# print(f"CNN AUC-ROC: {roc_auc_score(y_test, y_prob_cnn):.4f}")

In [2]:
import numpy as np
import pandas as pd
import cv2
import os
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, confusion_matrix, f1_score
from sklearn.linear_model import LogisticRegression
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, BatchNormalization, Dropout, Conv2D, MaxPooling2D, Flatten, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.regularizers import l2
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import make_pipeline
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical

# Constants
IMG_SIZE = (224, 224)  
BATCH_SIZE = 32
RANDOM_STATE = 42
EPOCHS = 50

# Paths
IMAGES_FOLDER = "/kaggle/input/glaucoma-datasets/G1020/Images"
df = pd.read_csv("/kaggle/input/glaucoma-datasets/G1020/G1020.csv")

# Improved F1 metric
def f1_metric(y_true, y_pred):
    y_true = K.cast(y_true, 'float32')
    y_pred = K.cast(K.round(y_pred), 'float32')
    
    tp = K.sum(y_true * y_pred)
    fp = K.sum((1 - y_true) * y_pred)
    fn = K.sum(y_true * (1 - y_pred))

    precision = tp / (tp + fp + K.epsilon())
    recall = tp / (tp + fn + K.epsilon())

    f1 = 2 * precision * recall / (precision + recall + K.epsilon())
    return f1

# Enhanced data loading with CLAHE and normalization
def load_data(df, IMG_SIZE):
    images = []
    labels = []
    
    for _, row in df.iterrows():
        img_name = row['imageID']
        label = row['binaryLabels']
        img_path = os.path.join(IMAGES_FOLDER, img_name)
        if os.path.exists(img_path):
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, IMG_SIZE)
            
            # Enhanced preprocessing
            lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
            l, a, b = cv2.split(lab)
            clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
            cl = clahe.apply(l)
            limg = cv2.merge((cl,a,b))
            img = cv2.cvtColor(limg, cv2.COLOR_LAB2RGB)
            
            # Normalize per image
            img = (img - img.min()) / (img.max() - img.min())
            
            images.append(img)
            labels.append(label)
    
    return np.array(images), np.array(labels)

# Load and preprocess data
images, labels = load_data(df, IMG_SIZE)
images = images.astype('float32')
labels = labels.astype('float32')

# Split data with stratification
X_train, X_test, y_train, y_test = train_test_split(
    images, labels, test_size=0.15, stratify=labels, random_state=RANDOM_STATE
)

# Enhanced augmentation
train_datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode='reflect'
)

# Feature extraction with EfficientNetB0
def extract_features(images):
    base_model = EfficientNetB0(weights="imagenet", include_top=False, input_shape=(*IMG_SIZE, 3))
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(1024, activation='relu', kernel_regularizer=l2(1e-4))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    feature_extractor = Model(inputs=base_model.input, outputs=x)
    
    features = feature_extractor.predict(images, batch_size=BATCH_SIZE, verbose=1)
    return features

print("Extracting training features...")
train_features = extract_features(X_train)
print("Extracting test features...")
test_features = extract_features(X_test)

# Normalize features
scaler = StandardScaler()
X_train_features = scaler.fit_transform(train_features)
X_test_features = scaler.transform(test_features)

# Improved class balancing
sampler = make_pipeline(
    SMOTE(sampling_strategy=0.8, random_state=RANDOM_STATE),
    RandomUnderSampler(sampling_strategy=0.9, random_state=RANDOM_STATE)
)
X_train_bal, y_train_bal = sampler.fit_resample(X_train_features, y_train)

# Enhanced ensemble model
estimators = [
    ('rf', RandomForestClassifier(n_estimators=500, max_depth=20, 
                                class_weight='balanced', random_state=RANDOM_STATE)),
    ('svm', SVC(C=1, gamma='scale', kernel='rbf', 
               probability=True, class_weight='balanced', random_state=RANDOM_STATE)),
    ('gb', GradientBoostingClassifier(n_estimators=300, learning_rate=0.05,
                                    max_depth=7, random_state=RANDOM_STATE))
]

stack = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression(class_weight='balanced', C=0.1, penalty='l2', max_iter=1000),
    cv=5,
    n_jobs=-1
)

stack.fit(X_train_bal, y_train_bal)
y_pred_stack = stack.predict(X_test_features)
y_prob_stack = stack.predict_proba(X_test_features)[:, 1]

# Enhanced CNN Model
def create_cnn_model():
    inputs = Input(shape=(*IMG_SIZE, 3))
    
    # Feature extraction
    x = Conv2D(32, (3,3), activation='relu', padding='same')(inputs)
    x = BatchNormalization()(x)
    x = MaxPooling2D(2,2)(x)
    x = Dropout(0.2)(x)
    
    x = Conv2D(64, (3,3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(2,2)(x)
    x = Dropout(0.3)(x)
    
    x = Conv2D(128, (3,3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(2,2)(x)
    x = Dropout(0.4)(x)
    
    # Classification head
    x = Flatten()(x)
    x = Dense(256, activation='relu', kernel_regularizer=l2(1e-4))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    outputs = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    
    model.compile(
        optimizer=Adam(learning_rate=1e-4),
        loss='binary_crossentropy',
        metrics=['accuracy', f1_metric]
    )
    return model

# Enhanced callbacks
callbacks = [
    EarlyStopping(patience=15, restore_best_weights=True, monitor='val_f1_metric', mode='max'),
    ReduceLROnPlateau(factor=0.2, patience=7, monitor='val_f1_metric', mode='max', min_lr=1e-6),
    ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_f1_metric', mode='max')
]

# Data generators for CNN
train_generator = train_datagen.flow(
    X_train, y_train,
    batch_size=BATCH_SIZE,
    shuffle=True
)

cnn_model = create_cnn_model()
history = cnn_model.fit(
    train_generator,
    steps_per_epoch=len(X_train) // BATCH_SIZE,
    validation_data=(X_test, y_test),
    epochs=EPOCHS,
    callbacks=callbacks,
    class_weight={0:1, 1:3}  # Higher weight for glaucoma cases
)

# Evaluate models
print("\n=== Enhanced Ensemble Model (Stacking) ===")
print(classification_report(y_test, y_pred_stack, target_names=["Normal", "Glaucoma"]))
print(f"Accuracy: {accuracy_score(y_test, y_pred_stack):.4f}")
print(f"AUC-ROC: {roc_auc_score(y_test, y_prob_stack):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred_stack, average='weighted'):.4f}")
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_stack))

# CNN evaluation
y_pred_cnn = (cnn_model.predict(X_test) > 0.5).astype(int)
y_prob_cnn = cnn_model.predict(X_test)
print("\n=== Enhanced CNN Model ===")
print(classification_report(y_test, y_pred_cnn, target_names=["Normal", "Glaucoma"]))
print(f"Accuracy: {accuracy_score(y_test, y_pred_cnn):.4f}")
print(f"AUC-ROC: {roc_auc_score(y_test, y_prob_cnn):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred_cnn, average='weighted'):.4f}")
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_cnn))

Extracting training features...
16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 14s 240ms/step
Extracting test features...
5/5 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


27/27 ━━━━━━━━━━━━━━━━━━━━ 28s 599ms/step - accuracy: 0.4805 - f1_metric: 0.3620 - loss: 1.8116 - val_accuracy: 0.2876 - val_f1_metric: 0.4413 - val_loss: 1.7369 - learning_rate: 1.0000e-04
Epoch 2/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6250 - f1_metric: 0.4667 - loss: 1.3684 - val_accuracy: 0.2876 - val_f1_metric: 0.4413 - val_loss: 1.8114 - learning_rate: 1.0000e-04
Epoch 3/50


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


27/27 ━━━━━━━━━━━━━━━━━━━━ 10s 286ms/step - accuracy: 0.4701 - f1_metric: 0.3959 - loss: 1.5950 - val_accuracy: 0.2876 - val_f1_metric: 0.4413 - val_loss: 4.1810 - learning_rate: 1.0000e-04
Epoch 4/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4375 - f1_metric: 0.3817 - loss: 2.4756 - val_accuracy: 0.2876 - val_f1_metric: 0.4413 - val_loss: 4.2784 - learning_rate: 1.0000e-04
Epoch 5/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 10s 269ms/step - accuracy: 0.4590 - f1_metric: 0.3888 - loss: 1.6756 - val_accuracy: 0.2876 - val_f1_metric: 0.4413 - val_loss: 5.8344 - learning_rate: 1.0000e-04
Epoch 6/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5000 - f1_metric: 0.5526 - loss: 1.4343 - val_accuracy: 0.2876 - val_f1_metric: 0.4413 - val_loss: 5.9107 - learning_rate: 1.0000e-04
Epoch 7/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 10s 283ms/step - accuracy: 0.4713 - f1_metric: 0.3717 - loss: 1.5069 - val_accuracy: 0.2876 - val_f1_metric: 0.4413 - val_loss: 8.2443 - learning_rate: 1.0000e-04
Epoch 8/50
2

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 107ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 

=== Enhanced CNN Model ===
              precision    recall  f1-score   support

      Normal       0.00      0.00      0.00       109
    Glaucoma       0.29      1.00      0.45        44

    accuracy                           0.29       153
   macro avg       0.14      0.50      0.22       153
weighted avg       0.08      0.29      0.13       153

Accuracy: 0.2876
AUC-ROC: 0.5273
F1 Score: 0.1285
Confusion Matrix:
[[  0 109]
 [  0  44]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [3]:
import os
import cv2
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, confusion_matrix, f1_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, BatchNormalization, Dropout, Conv2D, MaxPooling2D, Flatten, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

# Constants
IMG_SIZE = (224, 224)
BATCH_SIZE = 8
RANDOM_STATE = 42
EPOCHS = 50

# Custom F1 metric
def f1_metric(y_true, y_pred):
    y_true = K.cast(y_true, 'float32')
    y_pred = K.cast(K.round(y_pred), 'float32')
    tp = K.sum(y_true * y_pred)
    fp = K.sum((1 - y_true) * y_pred)
    fn = K.sum(y_true * (1 - y_pred))
    precision = tp / (tp + fp + K.epsilon())
    recall = tp / (tp + fn + K.epsilon())
    f1 = 2 * precision * recall / (precision + recall + K.epsilon())
    return f1

def load_images_from_folders(base_path):
    images = []
    labels = []
    
    class_mapping = {'normal': 0, 'glaucoma': 1}
    found_folders = {}

    # Walk recursively to find class folders
    for root, dirs, files in os.walk(base_path):
        for folder in dirs:
            folder_lower = folder.lower()
            if folder_lower in class_mapping:
                full_path = os.path.join(root, folder)
                found_folders[full_path] = class_mapping[folder_lower]

    if not found_folders:
        raise ValueError(f"No valid class folders ('normal', 'glaucoma') found in {base_path}")

    print(f"Found class folders: {list(found_folders.keys())}")

    for class_path, class_label in found_folders.items():
        for fname in os.listdir(class_path):
            if fname.lower().endswith(('.jpg', '.png', '.jpeg')):
                path = os.path.join(class_path, fname)
                try:
                    img = cv2.imread(path)
                    if img is None:
                        print(f"Warning: Could not read image {path}")
                        continue
                        
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    img = cv2.resize(img, IMG_SIZE)

                    # Enhanced preprocessing
                    lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
                    l, a, b = cv2.split(lab)
                    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
                    cl = clahe.apply(l)
                    limg = cv2.merge((cl, a, b))
                    img = cv2.cvtColor(limg, cv2.COLOR_LAB2RGB)

                    # Normalize and enhance contrast
                    img = img.astype('float32') / 255.0
                    img = (img - img.mean()) / (img.std() + 1e-7)

                    images.append(img)
                    labels.append(class_label)
                except Exception as e:
                    print(f"Error processing {path}: {str(e)}")
                    continue
    
    if len(images) == 0:
        raise ValueError(f"No valid images found in {base_path}")
    
    return np.array(images), np.array(labels)

# Define paths based on your structure
train_path = "/kaggle/input/drishtigs-retina-dataset-for-onh-segmentation/Training-20211018T055246Z-001/Training"
test_path = "/kaggle/input/drishtigs-retina-dataset-for-onh-segmentation/Test-20211018T060000Z-001/Test"

# Load data with enhanced error handling
try:
    X_train, y_train = load_images_from_folders(train_path)
    X_test, y_test = load_images_from_folders(test_path)
    print(f"Loaded {len(X_train)} training images and {len(X_test)} test images")
    print("Training class distribution:", np.unique(y_train, return_counts=True))
    print("Test class distribution:", np.unique(y_test, return_counts=True))
    
    # Check if we have at least 2 classes
    if len(np.unique(y_train)) < 2 or len(np.unique(y_test)) < 2:
        raise ValueError("Dataset must contain both normal and glaucoma cases for classification")
        
except Exception as e:
    print(f"\nError loading data: {str(e)}")
    print("\nDirectory structure:")
    for root, dirs, files in os.walk("/kaggle/input"):
        print(root)
    raise

# Data augmentation
train_datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='reflect'
)

# Feature extraction with EfficientNetB0
def extract_features(images):
    base_model = EfficientNetB0(weights="imagenet", include_top=False, input_shape=(*IMG_SIZE, 3))
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    feature_model = Model(inputs=base_model.input, outputs=x)
    return feature_model.predict(images, batch_size=BATCH_SIZE, verbose=1)

# Extract features
print("\nExtracting training features...")
train_features = extract_features(X_train)
print("Extracting test features...")
test_features = extract_features(X_test)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(train_features)
X_test_scaled = scaler.transform(test_features)

# CNN Model
def create_cnn_model():
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(*IMG_SIZE, 3)),
        BatchNormalization(),
        MaxPooling2D(2,2),
        Dropout(0.2),
        
        Conv2D(64, (3,3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(2,2),
        Dropout(0.3),
        
        Conv2D(128, (3,3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(2,2),
        Dropout(0.4),
        
        Flatten(),
        Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        BatchNormalization(),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(
        optimizer=Adam(learning_rate=1e-4),
        loss='binary_crossentropy',
        metrics=['accuracy', f1_metric]
    )
    return model

# Callbacks
callbacks = [
    EarlyStopping(patience=15, restore_best_weights=True, monitor='val_f1_metric', mode='max'),
    ReduceLROnPlateau(factor=0.2, patience=7, monitor='val_f1_metric', mode='max', min_lr=1e-6),
    ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_f1_metric', mode='max')
]

# Train CNN
print("\nTraining CNN model...")
cnn_model = create_cnn_model()
history = cnn_model.fit(
    train_datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
    steps_per_epoch=len(X_train) // BATCH_SIZE,
    validation_data=(X_test, y_test),
    epochs=EPOCHS,
    callbacks=callbacks,
    class_weight={0:1, 1:2}  # Adjust weights based on class imbalance
)

# Evaluate CNN
y_pred_cnn = (cnn_model.predict(X_test) > 0.5).astype(int)
y_prob_cnn = cnn_model.predict(X_test)
print("\n=== CNN Model Results ===")
print(classification_report(y_test, y_pred_cnn, target_names=["Normal", "Glaucoma"]))
print(f"Accuracy: {accuracy_score(y_test, y_pred_cnn):.4f}")
print(f"AUC: {roc_auc_score(y_test, y_prob_cnn):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred_cnn, average='weighted'):.4f}")
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_cnn))

# Traditional ML models
print("\nTraining traditional ML models...")
models = {
    'Random Forest': RandomForestClassifier(n_estimators=200, class_weight='balanced', random_state=RANDOM_STATE),
    'SVM': SVC(kernel='rbf', probability=True, class_weight='balanced', random_state=RANDOM_STATE),
    'Logistic Regression': LogisticRegression(class_weight='balanced', max_iter=1000, random_state=RANDOM_STATE)
}

for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    y_prob = model.predict_proba(X_test_scaled)[:, 1] if hasattr(model, "predict_proba") else None
    
    print(f"\n=== {name} Results ===")
    print(classification_report(y_test, y_pred, target_names=["Normal", "Glaucoma"]))
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    if y_prob is not None:
        print(f"AUC: {roc_auc_score(y_test, y_prob):.4f}")
    print(f"F1 Score: {f1_score(y_test, y_pred, average='weighted'):.4f}")
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))

Found class folders: ['/kaggle/input/drishtigs-retina-dataset-for-onh-segmentation/Training-20211018T055246Z-001/Training/Images/GLAUCOMA', '/kaggle/input/drishtigs-retina-dataset-for-onh-segmentation/Training-20211018T055246Z-001/Training/Images/NORMAL']
Found class folders: ['/kaggle/input/drishtigs-retina-dataset-for-onh-segmentation/Test-20211018T060000Z-001/Test/Images/glaucoma', '/kaggle/input/drishtigs-retina-dataset-for-onh-segmentation/Test-20211018T060000Z-001/Test/Images/normal']
Loaded 50 training images and 51 test images
Training class distribution: (array([0, 1]), array([18, 32]))
Test class distribution: (array([0, 1]), array([13, 38]))

Extracting training features...
7/7 ━━━━━━━━━━━━━━━━━━━━ 12s 973ms/step
Extracting test features...
7/7 ━━━━━━━━━━━━━━━━━━━━ 9s 835ms/step

Training CNN model...


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


6/6 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - accuracy: 0.6318 - f1_metric: 0.6422 - loss: 6.1627 - val_accuracy: 0.2353 - val_f1_metric: 0.0451 - val_loss: 5.7159 - learning_rate: 1.0000e-04
Epoch 2/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.6250 - f1_metric: 0.5556 - loss: 6.5801 - val_accuracy: 0.2353 - val_f1_metric: 0.0451 - val_loss: 5.6787 - learning_rate: 1.0000e-04
Epoch 3/50


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 383ms/step - accuracy: 0.5029 - f1_metric: 0.5507 - loss: 7.0009 - val_accuracy: 0.7451 - val_f1_metric: 0.7400 - val_loss: 5.3749 - learning_rate: 1.0000e-04
Epoch 4/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.5000 - f1_metric: 0.4687 - loss: 5.8687 - val_accuracy: 0.7451 - val_f1_metric: 0.7400 - val_loss: 5.3715 - learning_rate: 1.0000e-04
Epoch 5/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.4693 - f1_metric: 0.5558 - loss: 7.0287 - val_accuracy: 0.7451 - val_f1_metric: 0.7400 - val_loss: 5.5531 - learning_rate: 1.0000e-04
Epoch 6/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.0000e+00 - f1_metric: 0.5000 - loss: 6.8389 - val_accuracy: 0.7451 - val_f1_metric: 0.7400 - val_loss: 5.6036 - learning_rate: 1.0000e-04
Epoch 7/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 0.5101 - f1_metric: 0.5238 - loss: 6.6016 - val_accuracy: 0.7451 - val_f1_metric: 0.7400 - val_loss: 5.8708 - learning_rate: 1.0000e-04
Epoch 8/50
6/6 ━━━━━━

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



=== Random Forest Results ===
              precision    recall  f1-score   support

      Normal       0.00      0.00      0.00        13
    Glaucoma       0.75      1.00      0.85        38

    accuracy                           0.75        51
   macro avg       0.37      0.50      0.43        51
weighted avg       0.56      0.75      0.64        51

Accuracy: 0.7451
AUC: 0.4909
F1 Score: 0.6363
Confusion Matrix:
[[ 0 13]
 [ 0 38]]

=== SVM Results ===
              precision    recall  f1-score   support

      Normal       0.00      0.00      0.00        13
    Glaucoma       0.75      1.00      0.85        38

    accuracy                           0.75        51
   macro avg       0.37      0.50      0.43        51
weighted avg       0.56      0.75      0.64        51

Accuracy: 0.7451
AUC: 0.5000
F1 Score: 0.6363
Confusion Matrix:
[[ 0 13]
 [ 0 38]]

=== Logistic Regression Results ===
              precision    recall  f1-score   support

      Normal       0.25      1.00   

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif